## Generate list of unique city, state combos with expected price data
## Pull json coordinates of cities and save

In [6]:
import pandas as pd
import numpy as np
import requests
import pickle
import progressbar

In [ ]:
pd.set_option("display.max_columns",None)
cars = pd.read_pickle('../cars.pkl')
display(cars)

In [ ]:
# need a dictionary to get full state names
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

us_state_abbrev = {abbrev: full for full, abbrev in us_state_abbrev.items()}

In [ ]:
# create a city, state column
cars['state_full'] = cars['state'].map(us_state_abbrev)
cars['citystate'] = cars[['city', 'state_full']].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# only get non-null expected price rows
good_cars = cars[cars.expected_price.notnull()]

In [ ]:
# Generate and display list city, state combo that have cars with expected price data

citystate = good_cars['citystate'].unique()
print(f'The number of unique city, state combos is: {len(citystate)}')
print()
for combo in citystate:
    print(combo)

In [ ]:
# Use city state combo to get polygons from openstreemap

lefturl = 'https://nominatim.openstreetmap.org/search.php?q='
righturl = '&polygon_geojson=1&viewbox=&format=json'

# example with 1 combo
middurl = '+'.join(citystate[0].split(' '))

fullurl = lefturl + middurl + righturl
fullurl

In [ ]:
# use requests to get json data

resp = requests.get(fullurl)
resp.json()

In [ ]:
# generate a list of all urls with list comprehension

fullurls = [lefturl + '+'.join(middurl.split(' ')) + righturl for middurl in citystate]
fullurls = dict(zip(citystate, fullurls))
fullurls

In [ ]:
# get coordinates for each city and state
resave = 0

if resave:
    with progressbar.ProgressBar(maxval=len(fullurls)) as bar:
        city_state_json = {}
        i = 0
        for loc, url in fullurls.items():
            city_state_json[loc] = requests.get(url).json()
            i += 1
            bar.update(i)
    # save location information as pickle
    with open('city_coordinates.pkl', 'wb') as handle:
        pickle.dump(city_state_json,handle)
    # Test the loading
    with open('city_coordinates.pkl','rb') as handle:
        loaded_data = pickle.load(handle)
    loaded_data
    print(city_state_json == loaded_data)

## Try getting zip code coordinates 
#### https://pypi.org/project/uszipcode/



In [ ]:
# try with zip code module
from uszipcode import SearchEngine # pip install uszipcode

search = SearchEngine()

all_zips = good_cars['zip_code'].astype('int').unique()
coord = search.by_zipcode('84003').to_dict()['lat']
zipdict = {zipcode: [search.by_zipcode(zipcode).to_dict()['lat'], 
                     search.by_zipcode(zipcode).to_dict()['lng']] for zipcode in all_zips}
zipdict

## Another way to get zip code coordinates
https://examples.opendatasoft.com/api/v1/console/datasets/1.0/search/
https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?q=

In [ ]:
# try with opendatasoft api
left_url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q='
right_url = '&facet=state&facet=timezone&facet=dst'

zipcode = '84003'

test = requests.get(left_url + zipcode + right_url)
test.json()['records'][0]['fields']['geopoint']

In [ ]:
resave = 0

if resave:
    zipdict_api = {}
    with progressbar.ProgressBar(maxval=len(all_zips)) as bar:
        i = 0
        for zipp in all_zips:
            full_url = left_url+str(zipp)+right_url
            response = requests.get(full_url)
            try:
                loc = response.json()['records'][0]['fields']['geopoint']
            except:
                loc = [None, None]
            zipdict_api.update({zipp: loc})
            i += 1
            bar.update(i)
    # save
    with open('zip_coord_api.pkl', 'wb') as handle:
        pickle.dump(zipdict_api,handle)
    # Test the loading
    with open('zip_coord_api.pkl','rb') as handle:
        loaded_data = pickle.load(handle)
    print(zipdict_api == loaded_data)
    loaded_data

## Some coordinates are missing in both datasets, but luckily there aren't any missing in both

In [ ]:
# fill in missing coordinates
with open('zip_coord_api.pkl','rb') as handle:
    zipdict_api = pickle.load(handle)
    
for zipp in zipdict_api:
    missing = []
    if (None in zipdict[zipp]) and (None in zipdict_api[zipp]):
        print(f'Missing {zipp} in both dicts')
        missing.append(zipp)
    if (None in zipdict[zipp]):
        print(f'Missing {zipp} in zipdict')
    if (None in zipdict_api[zipp]): # Has more accurate coordinates, so fill in with this
        print(f'Missing {zipp} in zipdict_api')
        zipdict_api[zipp] = zipdict[zipp]

resave = 0
if resave:
    with open('zip_coord_api.pkl', 'wb') as handle:
        pickle.dump(zipdict_api, handle)

zipdict_api

# ipyleaflet

Kind of like a free version of Google Maps API

In [4]:
import json
from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl, CircleMarker, LayerGroup
# if above module isn't installed, do BOTH of the following:
# pip install ipyleaflet
# jupyter nbextension enable --py --sys-prefix ipyleaflet

from ipywidgets import HTML

from polylabel import polylabel # if not installed, do pip install python-polylabel

In [ ]:
# Example from https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html

center = [38.128, 2.588]
zoom = 5

Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

In [ ]:
# now load a more local map

center = [40.7608, -111.8910] # centered on SLC, UT
zoom = 6

Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

In [ ]:
# now let's make an interactive hoverable map!

# play around with json polygon data for St. George and Hurricane, UT

# instructions for where to get geojson data: https://gis.stackexchange.com/questions/183248/getting-polygon-boundaries-of-city-in-json-from-google-maps-api
# specifically the bit about adding '&format=json' to search url

# center = [40.7608, -111.8910] # SLC, UT
# center = [39.7102, -111.8363] # Nephi, UT
center = [37.0965, -113.5684] # St. George, UT
zoom = 9
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m.add_control(FullScreenControl())

# parse St. George data
with open('polygons\st_george_ut.json') as city_file:
    city_data = json.load(city_file)
city_data_1 = city_data[-1]['geojson']
city_data_1['varinput'] = 'mystring'

# parse Hurricane data
with open('polygons\hurricane_ut.json') as city_file:
    city_data = json.load(city_file)
city_data_2 = city_data[-1]['geojson']
city_data_2['varinput'] = 'mystring2'
    
def tmpfunc(event, feature):
#     print(event)
    
    # remove old popup layer
    if isinstance(m.layers[-1], Popup):
        m.remove_layer(m.layers[-1])
    
    # get center of current polygon
    txt_loc = polylabel(feature['coordinates'])[::-1]

    # add a popup layer on hover over a city
    message = HTML()
    message.value = feature['varinput']
    popup = Popup(location=txt_loc, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

    m.add_layer(popup) # add the new layer

# create the GeoJSON layers (polygons overlaying each city)
geo_json_1 = GeoJSON(data=city_data_1, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.3})
geo_json_2 = GeoJSON(data=city_data_2, style = {'color': 'orange', 'opacity':1, 'weight':1.9, 'fillOpacity':0.3})
geo_json_1.on_hover(tmpfunc)
geo_json_2.on_hover(tmpfunc)
m.add_layer(geo_json_1)
m.add_layer(geo_json_2)

m

In [ ]:
# layer group example from: https://ipyleaflet.readthedocs.io/en/latest/api_reference/layer_group.html#
from ipyleaflet import (
    Map, basemaps, basemap_to_tiles,
    Circle, Marker, Rectangle, LayerGroup
)

toner = basemap_to_tiles(basemaps.Stamen.Toner)

m = Map(layers=(toner, ), center=(50, 354), zoom=5)

# Create some layers
marker = Marker(location=(50, 354))
circle = Circle(location=(50, 370), radius=50000, color="yellow", fill_color="yellow")
rectangle = Rectangle(bounds=((54, 354), (55, 360)), color="orange", fill_color="orange")

# Create layer group
layer_group = LayerGroup(layers=(marker, circle))

m.add_layer(layer_group)

layer_group.add_layer(rectangle)

# layer_group.remove_layer(circle)

m

# Let's try making a map for each city/state combo!

Could change circle size by number of cars posted, and color by how good the deal is.

In [1]:
# experimenting with colors...
from matplotlib import cm
from matplotlib.colors import rgb2hex
import random
coolwarm = cm.get_cmap('coolwarm',5)
rgb2hex(coolwarm(0.5))
random.randint(0,4)

0

In [8]:
colors = ['#d7191c','#fdae61','#ffffbf','#a6d96a','#1a9641']
colors = ['red', 'orange', 'green']

In [9]:
utah_center = [39.3210, -111.0937]
zoom = 6
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=utah_center, zoom=zoom)
m.add_control(FullScreenControl())

# load zip code coordinates
with open('zip_coord_api.pkl','rb') as handle:
    zip_coord = pickle.load(handle)

# create a layer group 
layer_group = LayerGroup()
for zipp, coord in zip_coord.items():
    circle = CircleMarker()
    circle.location = coord
    circle.radius = random.randint(1,10)
    circle.weight = 2
    circle.opacity = 0.8
    color = random.randint(0,2)
    circle.color = colors[color]
    circle.fill_color = colors[color]
    circle.fill_opacity = 0.3
    layer_group.add_layer(circle)
    

m.add_layer(layer_group)
m

Map(center=[39.321, -111.0937], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

# Gmaps (Google Maps)

Note: to avoid the watermark, you have to sign up for a billing service for your API account as of June 2018

In [ ]:
# Example from gmaps documentation at https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

import gmaps
import gmaps.datasets

In [ ]:
earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
earthquake_df.head()

In [ ]:
locations = earthquake_df[['latitude', 'longitude']]
weights = earthquake_df['magnitude']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

In [ ]:
new_york_coordinates = (40.75, -74.00)
gmaps.figure(center=new_york_coordinates, zoom_level=12)